We will use CatBoost as base model. At first, let's try to put data to the model without transformations. After that we will try to replace missing values by -1. After that we will try to fill gaps with KNNImputer from Scikit Learn. Next idea to try is filling the missing values with average values depending on class labels.

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import missingno as msno
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
from numpy.lib.function_base import average
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
# from imblearn.over_sampling import SMOTE

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [78]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from catboost import CatBoostClassifier

In [79]:
#pip install imbalanced-learn
#pip install missingno
#pip install catboost

In [5]:
from matplotlib import rcParams

# figure size in inches
rcParams['figure.figsize'] = 15,12

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [6]:
data = pd.read_csv('../CSV/Norwey data.csv', sep=';')
# data

In [7]:
features = data.loc[:, "CALI":"RXO"]
# features

In [8]:
targets = data['FORCE_2020_LITHOFACIES_LITHOLOGY']

label_encoder = preprocessing.LabelEncoder()
encoded_targets = label_encoder.fit_transform(targets)

In [9]:
gaps_pct_threshold = 50

# Percent of missing values in data
nans_pct = (features.isnull().sum()/features.shape[0]*100).astype('int').sort_values(ascending=False)
# We select features with missing values less than n %
good_columns = nans_pct[nans_pct < gaps_pct_threshold].index.to_list()
good_features = features[good_columns]
good_targets = encoded_targets[good_features.index]

In [11]:
def fill_nans_with_value_from_distribution(data, target):
    data_n_target = data.join(target)
    target_col_name = target.name
    # for class_name in np.unique(target):
    for col_name in data.columns:
        data_n_target[col_name] = data_n_target[col_name].fillna(data_n_target.groupby(target_col_name)[col_name].transform('mean'))
    return data_n_target.drop(columns=[target_col_name])

**1. CatBoost and Virgin Data**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, encoded_targets, test_size=0.3, random_state=1, shuffle=True, stratify=encoded_targets)

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train, y_train, verbose=False)

score_1 = catboost_model.score(X_test, y_test)

print(f"Score: {score_1}")

Score: 0.7648723921698173


**2. CatBoost + normalized data and replaced missing values**

In [ ]:
scaler = preprocessing.StandardScaler()

imputer = SimpleImputer(strategy='constant', fill_value=-1)

X_train, X_test, y_train, y_test = train_test_split(features, encoded_targets, random_state=0)

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

pipeline_2 = Pipeline(steps=[('scaler', scaler), ('imputer', imputer), ('model', catboost_model)])

pipeline_2.fit(X_train, y_train)

score_2 = pipeline_2.score(X_test, y_test)

print(f"Score: {score_2}")

0:	learn: 1.2678461	total: 1.56s	remaining: 1m 47s
1:	learn: 1.1104466	total: 2.81s	remaining: 1m 35s
2:	learn: 1.0550017	total: 4s	remaining: 1m 29s
3:	learn: 0.9970682	total: 5.34s	remaining: 1m 28s
4:	learn: 0.9616488	total: 6.73s	remaining: 1m 27s
5:	learn: 0.9410390	total: 7.99s	remaining: 1m 25s
6:	learn: 0.9259078	total: 9.26s	remaining: 1m 23s
7:	learn: 0.9116644	total: 10.6s	remaining: 1m 21s
8:	learn: 0.9120892	total: 11.7s	remaining: 1m 19s
9:	learn: 0.8929877	total: 13.8s	remaining: 1m 22s
10:	learn: 0.8782309	total: 15.4s	remaining: 1m 22s
11:	learn: 0.8694416	total: 16.8s	remaining: 1m 21s
12:	learn: 0.8624586	total: 18.2s	remaining: 1m 19s
13:	learn: 0.8527893	total: 19.7s	remaining: 1m 18s
14:	learn: 0.8716393	total: 20.8s	remaining: 1m 16s
15:	learn: 0.8491921	total: 21.9s	remaining: 1m 13s
16:	learn: 0.8407420	total: 23.6s	remaining: 1m 13s
17:	learn: 0.8344396	total: 24.8s	remaining: 1m 11s
18:	learn: 0.8307386	total: 26s	remaining: 1m 9s
19:	learn: 0.8231496	total: 

We did not get significantly high score from both ideas. Let's move further

**3. CatBoost + Features that have less than 50% of missing values**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(good_features, good_targets, test_size=0.3, random_state=1, shuffle=True, stratify=encoded_targets)

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train, y_train, verbose=False)

score_3 = catboost_model.score(X_test, y_test)

print(f"Score: {score_3}")

Score: 0.7630840030300097


Removing of features with huge number of missing values decreases the score

**4. CatBoost and Stochastic KNNImputer**

In [ ]:
def stochastic_knn_imputer(knn_imputer, data, num_butches, fit_or_transform):
    data_after_knn_processing = None
    previous_chunk = None
    for chunk in tqdm(np.array_split(data, num_butches)):   # 24579 rows in chunk for X_train
        if previous_chunk is not None:
            chunk = np.concatenate((previous_chunk, chunk))
            previous_chunk = None
        if ((np.isnan(chunk).sum(axis=0)/chunk.shape[0]*100)==100).any():   # if any column in chunk has 100% nans then we concatenate it with next cbunk 
            previous_chunk = chunk
            continue
        imputted_chunk = knn_imputer.fit_transform(chunk) if fit_or_transform == 'fit' else knn_imputer.transform(chunk)
        if data_after_knn_processing is None:
            data_after_knn_processing = imputted_chunk
        else:
            data_after_knn_processing = np.concatenate((data_after_knn_processing, imputted_chunk))
    return data_after_knn_processing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(good_features, good_targets, test_size=0.3, random_state=1, shuffle=False)

imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean', missing_values=np.nan)

# fit stochastic knn imputer
X_train_imputted = stochastic_knn_imputer(imputer, X_train, 100, 'fit')
X_test_impitted = stochastic_knn_imputer(imputer, X_test, 14, 'transform')

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train_imputted, y_train)

score_4 = catboost_model.score(X_test_impitted, y_test)
print(f"Score: {score_4}")

  0%|          | 0/10 [00:00<?, ?it/s]

**5. CatBoost + Filling missing values with average values depending on a class label**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(good_features, good_targets, test_size=0.3, random_state=1, shuffle=True, stratify=encoded_targets)

prepared_train_target = pd.Series(y_train, index=X_train.index, name=targets.name)
prepared_test_target = pd.Series(y_test, index=X_test.index, name=targets.name)

X_train_filled = fill_nans_with_value_from_distribution(X_train, prepared_train_target)
X_train_filled = X_train_filled.fillna(X_train_filled.mean())
y_train_prepared = prepared_train_target[X_train_filled.index]

X_test_filled = fill_nans_with_value_from_distribution(X_test, prepared_test_target)
X_test_filled = X_test_filled.fillna(X_test_filled.mean())
y_test_prepared = prepared_test_target[X_test_filled.index]

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train_filled, y_train_prepared, verbose=False)

score_5 = catboost_model.score(X_test_filled, y_test_prepared)

print(f"Score: {score_5}")

Score: 0.9092762719490594


Filling NANs with the average value depending on class label helps to improve data significantly. Score rises from .77 to .90

**6. Data Oversampling** <br>
We have minority class with 103 samples. Let us oversample data to help the model to get more info from data

In [ ]:
pd.Series(good_targets).value_counts()

1     720803
0     168937
2     150455
3      56320
6      33329
11     15245
4      10513
8       8213
9       3820
5       1688
7       1085
10       103
dtype: int64

In [ ]:
strategy = {10:400}  # increasing the minority class size
oversample = SMOTE(sampling_strategy=strategy)
X_train_oversampled, y_train_oversampled = oversample.fit_resample(X_train_filled, y_train_prepared)

catboost_model = CatBoostClassifier(iterations=70,
                           learning_rate=0.5,
                           depth=2,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train_oversampled, y_train_oversampled, verbose=False)

score_6 = catboost_model.score(X_test_filled, y_test_prepared)

print(f"Score: {score_6}")

Score: 0.9044863507179186


In [ ]:
pd.Series(y_train_oversampled).value_counts()

1     504562
0     118256
2     105318
3      39424
6      23330
11     10671
4       7359
8       5749
9       2674
5       1182
7        760
10       300
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

Oversampling method did not help us to improve the data.

**Model tuning**

In [13]:
pip install scikit-optimize

Note: you may need to restart the kernel to use updated packages.


In [37]:
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

data = (X_train_filled, y_train_prepared, X_test_filled, y_test_prepared)

space = [Integer(100, 500, name='iterations'),
         Real(0.01, 1, 'log-uniform', name='learning_rate'),
         Integer(2, 10, name='depth'),
         Integer(1, 10, name='l2_leaf_reg'),
         # Categorical(['TotalF1:average=Macro'], name='custom_metric'),
        ]

@use_named_args(space)
def objective(**params):
    X_train, y_train, X_test, y_test = data
    model = CatBoostClassifier(**params, custom_metric=['TotalF1:average=Macro'])
    model.fit(X_train, y_train)
    return -model.score(X_test, y_test)

result = gp_minimize(objective, space, n_calls=20, random_state=1, verbose=True, n_jobs=-2)
print('Best Score: %.3f' % (-result.fun))
print('Best Parameters:' + (" {:.5f}" * len(result.x)).format(*result.x))


Iteration No: 1 started. Evaluating function at random point.
0:	learn: 1.1549213	total: 399ms	remaining: 3m 18s
1:	learn: 8.4807190	total: 759ms	remaining: 3m 8s
2:	learn: 1009.5566510	total: 1.19s	remaining: 3m 16s
3:	learn: 765.0088340	total: 1.59s	remaining: 3m 17s
4:	learn: 765.8676411	total: 1.95s	remaining: 3m 12s
5:	learn: 705.4610237	total: 2.35s	remaining: 3m 12s
6:	learn: 1690.8897420	total: 2.73s	remaining: 3m 11s
7:	learn: 1526.6028286	total: 3.21s	remaining: 3m 17s
8:	learn: 1224.3346404	total: 3.63s	remaining: 3m 17s
9:	learn: 1458.7905626	total: 4.05s	remaining: 3m 18s
10:	learn: 1143.3737648	total: 4.46s	remaining: 3m 17s
11:	learn: 1078.8347870	total: 4.9s	remaining: 3m 18s
12:	learn: 1052.0748367	total: 5.34s	remaining: 3m 19s
13:	learn: 1005.6367855	total: 5.77s	remaining: 3m 19s
14:	learn: 957.6038069	total: 6.35s	remaining: 3m 24s
15:	learn: 1223.5934772	total: 6.82s	remaining: 3m 25s
16:	learn: 1199.6311109	total: 7.3s	remaining: 3m 27s
17:	learn: 1109.8417600	to

151:	learn: 192.2269063	total: 1m 4s	remaining: 2m 28s
152:	learn: 190.1199764	total: 1m 5s	remaining: 2m 27s
153:	learn: 189.5214387	total: 1m 5s	remaining: 2m 27s
154:	learn: 189.0971259	total: 1m 6s	remaining: 2m 26s
155:	learn: 188.0702277	total: 1m 6s	remaining: 2m 26s
156:	learn: 187.0423922	total: 1m 6s	remaining: 2m 25s
157:	learn: 186.4449128	total: 1m 7s	remaining: 2m 25s
158:	learn: 186.1710443	total: 1m 7s	remaining: 2m 24s
159:	learn: 184.9733594	total: 1m 8s	remaining: 2m 24s
160:	learn: 184.8326064	total: 1m 8s	remaining: 2m 23s
161:	learn: 183.9927676	total: 1m 8s	remaining: 2m 23s
162:	learn: 183.1825332	total: 1m 9s	remaining: 2m 22s
163:	learn: 182.7297208	total: 1m 9s	remaining: 2m 22s
164:	learn: 181.8326345	total: 1m 10s	remaining: 2m 22s
165:	learn: 181.4866552	total: 1m 10s	remaining: 2m 21s
166:	learn: 180.0211848	total: 1m 11s	remaining: 2m 21s
167:	learn: 179.4902251	total: 1m 11s	remaining: 2m 20s
168:	learn: 178.1268328	total: 1m 11s	remaining: 2m 20s
169:	

299:	learn: 129.3791890	total: 2m 5s	remaining: 1m 23s
300:	learn: 129.1358236	total: 2m 5s	remaining: 1m 22s
301:	learn: 128.9739922	total: 2m 6s	remaining: 1m 22s
302:	learn: 128.7103338	total: 2m 6s	remaining: 1m 21s
303:	learn: 128.6750667	total: 2m 6s	remaining: 1m 21s
304:	learn: 128.2856266	total: 2m 7s	remaining: 1m 21s
305:	learn: 128.1587864	total: 2m 7s	remaining: 1m 20s
306:	learn: 127.9923520	total: 2m 8s	remaining: 1m 20s
307:	learn: 127.8158914	total: 2m 8s	remaining: 1m 19s
308:	learn: 127.5697877	total: 2m 9s	remaining: 1m 19s
309:	learn: 127.3199948	total: 2m 9s	remaining: 1m 18s
310:	learn: 127.0352053	total: 2m 9s	remaining: 1m 18s
311:	learn: 126.8717474	total: 2m 10s	remaining: 1m 18s
312:	learn: 126.7104884	total: 2m 10s	remaining: 1m 17s
313:	learn: 126.5718335	total: 2m 10s	remaining: 1m 17s
314:	learn: 126.4124688	total: 2m 11s	remaining: 1m 16s
315:	learn: 126.3969472	total: 2m 11s	remaining: 1m 16s
316:	learn: 126.2340119	total: 2m 12s	remaining: 1m 15s
317:

449:	learn: 107.6351552	total: 3m 6s	remaining: 20.3s
450:	learn: 107.5989331	total: 3m 6s	remaining: 19.8s
451:	learn: 107.5809528	total: 3m 6s	remaining: 19.4s
452:	learn: 107.0823759	total: 3m 7s	remaining: 19s
453:	learn: 107.0162956	total: 3m 7s	remaining: 18.6s
454:	learn: 106.8157204	total: 3m 8s	remaining: 18.2s
455:	learn: 106.1132323	total: 3m 8s	remaining: 17.8s
456:	learn: 105.9216030	total: 3m 8s	remaining: 17.4s
457:	learn: 105.6447323	total: 3m 9s	remaining: 16.9s
458:	learn: 105.3044195	total: 3m 9s	remaining: 16.5s
459:	learn: 105.2445340	total: 3m 10s	remaining: 16.1s
460:	learn: 105.2316382	total: 3m 10s	remaining: 15.7s
461:	learn: 105.1990597	total: 3m 10s	remaining: 15.3s
462:	learn: 104.7964297	total: 3m 11s	remaining: 14.9s
463:	learn: 104.7590363	total: 3m 11s	remaining: 14.5s
464:	learn: 104.6834576	total: 3m 12s	remaining: 14s
465:	learn: 104.6632302	total: 3m 12s	remaining: 13.6s
466:	learn: 104.5948933	total: 3m 12s	remaining: 13.2s
467:	learn: 104.4873380	

103:	learn: 0.2059357	total: 50.8s	remaining: 44s
104:	learn: 0.2041522	total: 51.5s	remaining: 43.6s
105:	learn: 0.2026396	total: 52s	remaining: 43.2s
106:	learn: 0.2010274	total: 52.6s	remaining: 42.8s
107:	learn: 0.1996262	total: 53.2s	remaining: 42.4s
108:	learn: 0.1984051	total: 53.9s	remaining: 42s
109:	learn: 0.1970602	total: 54.5s	remaining: 41.6s
110:	learn: 0.1957778	total: 55s	remaining: 41.1s
111:	learn: 0.1945069	total: 55.5s	remaining: 40.7s
112:	learn: 0.1935556	total: 56s	remaining: 40.1s
113:	learn: 0.1918883	total: 56.5s	remaining: 39.7s
114:	learn: 0.1906593	total: 57s	remaining: 39.2s
115:	learn: 0.1896325	total: 57.5s	remaining: 38.6s
116:	learn: 0.1884391	total: 57.9s	remaining: 38.1s
117:	learn: 0.1865978	total: 58.5s	remaining: 37.7s
118:	learn: 0.1856338	total: 58.9s	remaining: 37.1s
119:	learn: 0.1845908	total: 59.3s	remaining: 36.6s
120:	learn: 0.1834077	total: 59.8s	remaining: 36.1s
121:	learn: 0.1820944	total: 1m	remaining: 35.6s
122:	learn: 0.1808772	total

...

299:	learn: 0.0262215	total: 12m 10s	remaining: 8m 6s
300:	learn: 0.0261123	total: 12m 12s	remaining: 8m 4s
301:	learn: 0.0260303	total: 12m 15s	remaining: 8m 2s
302:	learn: 0.0259484	total: 12m 17s	remaining: 7m 59s
303:	learn: 0.0259139	total: 12m 20s	remaining: 7m 57s
304:	learn: 0.0258584	total: 12m 22s	remaining: 7m 54s
305:	learn: 0.0257906	total: 12m 25s	remaining: 7m 52s
306:	learn: 0.0257298	total: 12m 27s	remaining: 7m 49s
307:	learn: 0.0256407	total: 12m 29s	remaining: 7m 47s
308:	learn: 0.0255741	total: 12m 32s	remaining: 7m 45s
309:	learn: 0.0255161	total: 12m 34s	remaining: 7m 42s
310:	learn: 0.0254434	total: 12m 37s	remaining: 7m 40s
311:	learn: 0.0253821	total: 12m 39s	remaining: 7m 37s
312:	learn: 0.0253232	total: 12m 41s	remaining: 7m 35s
313:	learn: 0.0252507	total: 12m 44s	remaining: 7m 32s
314:	learn: 0.0251853	total: 12m 46s	remaining: 7m 30s
315:	learn: 0.0251287	total: 12m 49s	remaining: 7m 27s
316:	learn: 0.0250569	total: 12m 51s	remaining: 7m 25s
317:	learn: 0

450:	learn: 0.0185619	total: 18m 15s	remaining: 1m 59s
451:	learn: 0.0185218	total: 18m 18s	remaining: 1m 56s
452:	learn: 0.0184871	total: 18m 20s	remaining: 1m 54s
453:	learn: 0.0184427	total: 18m 22s	remaining: 1m 51s
454:	learn: 0.0184128	total: 18m 25s	remaining: 1m 49s
455:	learn: 0.0183760	total: 18m 27s	remaining: 1m 46s
456:	learn: 0.0183438	total: 18m 30s	remaining: 1m 44s
457:	learn: 0.0183168	total: 18m 32s	remaining: 1m 42s
458:	learn: 0.0182844	total: 18m 35s	remaining: 1m 39s
459:	learn: 0.0182162	total: 18m 37s	remaining: 1m 37s
460:	learn: 0.0181783	total: 18m 40s	remaining: 1m 34s
461:	learn: 0.0181556	total: 18m 42s	remaining: 1m 32s
462:	learn: 0.0181027	total: 18m 44s	remaining: 1m 29s
463:	learn: 0.0180718	total: 18m 47s	remaining: 1m 27s
464:	learn: 0.0180437	total: 18m 49s	remaining: 1m 25s
465:	learn: 0.0180085	total: 18m 52s	remaining: 1m 22s
466:	learn: 0.0179561	total: 18m 54s	remaining: 1m 20s
467:	learn: 0.0179234	total: 18m 57s	remaining: 1m 17s
468:	learn

100:	learn: 0.3647763	total: 4m 5s	remaining: 16m 8s
101:	learn: 0.3610979	total: 4m 7s	remaining: 16m 6s
102:	learn: 0.3571840	total: 4m 10s	remaining: 16m 4s
103:	learn: 0.3536988	total: 4m 12s	remaining: 16m 1s
104:	learn: 0.3502255	total: 4m 14s	remaining: 15m 59s
105:	learn: 0.3470287	total: 4m 17s	remaining: 15m 56s
106:	learn: 0.3438089	total: 4m 19s	remaining: 15m 53s
107:	learn: 0.3406329	total: 4m 22s	remaining: 15m 51s
108:	learn: 0.3375746	total: 4m 24s	remaining: 15m 48s
109:	learn: 0.3343593	total: 4m 26s	remaining: 15m 45s
110:	learn: 0.3312551	total: 4m 29s	remaining: 15m 43s
111:	learn: 0.3280893	total: 4m 31s	remaining: 15m 40s
112:	learn: 0.3252536	total: 4m 33s	remaining: 15m 37s
113:	learn: 0.3220920	total: 4m 36s	remaining: 15m 35s
114:	learn: 0.3190816	total: 4m 38s	remaining: 15m 32s
115:	learn: 0.3161736	total: 4m 41s	remaining: 15m 30s
116:	learn: 0.3134918	total: 4m 43s	remaining: 15m 28s
117:	learn: 0.3106496	total: 4m 45s	remaining: 15m 25s
118:	learn: 0.30

251:	learn: 0.1366896	total: 10m 5s	remaining: 9m 55s
252:	learn: 0.1360885	total: 10m 7s	remaining: 9m 53s
253:	learn: 0.1355931	total: 10m 10s	remaining: 9m 51s
254:	learn: 0.1349978	total: 10m 12s	remaining: 9m 48s
255:	learn: 0.1344210	total: 10m 15s	remaining: 9m 46s
256:	learn: 0.1338954	total: 10m 17s	remaining: 9m 43s
257:	learn: 0.1333650	total: 10m 19s	remaining: 9m 41s
258:	learn: 0.1327802	total: 10m 22s	remaining: 9m 39s
259:	learn: 0.1321948	total: 10m 24s	remaining: 9m 36s
260:	learn: 0.1318155	total: 10m 26s	remaining: 9m 34s
261:	learn: 0.1311931	total: 10m 29s	remaining: 9m 31s
262:	learn: 0.1306413	total: 10m 31s	remaining: 9m 29s
263:	learn: 0.1300698	total: 10m 34s	remaining: 9m 27s
264:	learn: 0.1295369	total: 10m 36s	remaining: 9m 24s
265:	learn: 0.1289967	total: 10m 39s	remaining: 9m 22s
266:	learn: 0.1284352	total: 10m 42s	remaining: 9m 20s
267:	learn: 0.1280090	total: 10m 44s	remaining: 9m 18s
268:	learn: 0.1275003	total: 10m 47s	remaining: 9m 16s
269:	learn: 

401:	learn: 0.0858938	total: 16m 11s	remaining: 3m 56s
402:	learn: 0.0856942	total: 16m 14s	remaining: 3m 54s
403:	learn: 0.0855229	total: 16m 16s	remaining: 3m 52s
404:	learn: 0.0853308	total: 16m 19s	remaining: 3m 49s
405:	learn: 0.0850926	total: 16m 21s	remaining: 3m 47s
406:	learn: 0.0848865	total: 16m 24s	remaining: 3m 44s
407:	learn: 0.0846865	total: 16m 26s	remaining: 3m 42s
408:	learn: 0.0845325	total: 16m 28s	remaining: 3m 40s
409:	learn: 0.0843680	total: 16m 31s	remaining: 3m 37s
410:	learn: 0.0841551	total: 16m 33s	remaining: 3m 35s
411:	learn: 0.0839622	total: 16m 36s	remaining: 3m 32s
412:	learn: 0.0837890	total: 16m 38s	remaining: 3m 30s
413:	learn: 0.0836242	total: 16m 40s	remaining: 3m 27s
414:	learn: 0.0833829	total: 16m 43s	remaining: 3m 25s
415:	learn: 0.0832065	total: 16m 45s	remaining: 3m 23s
416:	learn: 0.0830640	total: 16m 48s	remaining: 3m 20s
417:	learn: 0.0829170	total: 16m 50s	remaining: 3m 18s
418:	learn: 0.0827473	total: 16m 52s	remaining: 3m 15s
419:	learn

In [39]:
catboost_model = CatBoostClassifier(iterations=500,
                           learning_rate=0.16,
                           depth=10,
                           l2_leaf_reg=6,
                           custom_metric=['TotalF1:average=Macro']
                           )

catboost_model.fit(X_train_filled, y_train_prepared, verbose=False)

score_7= catboost_model.score(X_test_filled, y_test_prepared)

print(f"Score: {score_7}")

Score: 0.9701270667570354


As conclusion, we can say that filling the missing values with average values depending on class label gave good result rising score on test from 77% to 91%. Model tuning rised the accuracy of prediction to 97%. <br>Unfortunately, oversampling did not increase the score on test. Methods which being used also are 1) CatBoost and Virgin Data, 2) CatBoost + normalized data and replaced missing values(-1), 3) removing features that contains more than 50% of missing values and 4) Stochastic KNN Imputer of gaps. 